# Many Students in Many Courses

**Instructions**

This application will read roster data in JSON format, parse the file, and then produce an SQLite database that contains a User, Course, and Member table and populate the tables from the data file.

You can base your solution on this code: http://www.py4e.com/code3/roster/roster.py - this code is incomplete as you need to modify the program to store the role column in the Member table to complete the assignment.

Each student gets their own file for the assignment. Download [this file](https://www.py4e.com/tools/sql-intro/roster_data.php?PHPSESSID=fb0cf97d46e2b6ec23454c01984988de) and save it as roster_data.json. Move the downloaded file into the same folder as your roster.py program.

Once you have made the necessary changes to the program and it has been run successfully reading the above JSON data, run the following SQL command:

```sql
SELECT User.name,Course.title, Member.role FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY User.name DESC, Course.title DESC, Member.role DESC LIMIT 2;
```

The output should look as follows:

```text
Zi|si110|0
Zennon|si430|0
```

Once that query gives the correct data, run this query:

```sql
SELECT 'XYZZY' || hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X LIMIT 1;
```

You should get one row with a string that looks like XYZZY53656C696E613333.

In [1]:
import json
import sqlite3
import pandas as pd

conn = sqlite3.connect('./data/rosterdb.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript(
                    '''
                    DROP TABLE IF EXISTS User;
                    DROP TABLE IF EXISTS Member;
                    DROP TABLE IF EXISTS Course;

                    CREATE TABLE User ( id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                                        name TEXT UNIQUE);

                    CREATE TABLE Course (   id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                                            title TEXT UNIQUE);

                    CREATE TABLE Member (   user_id INTEGER,
                                            course_id INTEGER,
                                            role INTEGER,
                                            PRIMARY KEY (user_id, course_id));
                    '''
                )

fname = 'roster_data.json'

str_data = open("./data/" + fname).read()
json_data = json.loads(str_data)

for entry in json_data:
    name = entry[0]
    title = entry[1]
    role = entry[2]

    cur.execute(
                '''
                INSERT INTO User (name)
                SELECT ? 
                WHERE NOT EXISTS (  SELECT * 
                                    FROM User 
                                    WHERE name = ?)
                ''', (name, name, )
                )
    
    cur.execute(
                '''
                SELECT id 
                FROM User 
                WHERE name = ? 
                ''', (name, )
                )
    user_id = cur.fetchone()[0]

    cur.execute(
                '''
                INSERT INTO Course (title)
                SELECT ? 
                WHERE NOT EXISTS (  SELECT * 
                                    FROM Course 
                                    WHERE title = ?)
                ''', (title, title, )
                )
    
    cur.execute(
                '''
                SELECT id 
                FROM Course
                WHERE title = ? 
                ''', (title, )
                )
    course_id = cur.fetchone()[0]

    cur.execute(
                '''
                INSERT INTO Member (user_id, course_id, role) 
                VALUES (?, ?, ?)
                ''', (user_id, course_id, role, )
                )
    
query_1 =   '''
            SELECT User.name, Course.title, Member.role 
            FROM User JOIN Member JOIN Course 
                ON User.id = Member.user_id AND Member.course_id = Course.id
            ORDER BY User.name DESC, Course.title DESC, Member.role DESC 
            LIMIT 2;
            '''

df_1 = pd.read_sql_query(query_1, conn)

query_2 =   '''
            SELECT 'XYZZY' || HEX(User.name || Course.title || Member.role ) AS X 
            FROM User JOIN Member JOIN Course 
                ON User.id = Member.user_id AND Member.course_id = Course.id
            ORDER BY X 
            LIMIT 1;
            '''

df_2 = pd.read_sql_query(query_2, conn)

conn.commit()
conn.close()

In [2]:
df_1

,name,title,role
0,Zi,si110,0
1,Zennon,si430,0


In [3]:
df_2

,X
0,XYZZY416172616E736931303630
